# Data Mining Versuch Recommender Systeme

* Autor: Prof. Dr. Johannes Maucher
* Datum: 30.09.2015

## Abgabe:

- **Abzugeben ist das Jupyter Notebook mit dem verlangten Implementierungen und den entsprechenden Ausgaben.**
- **Das Notebook ist als .ipynb und als .html abzugeben.**
- **Klausurelevante Fragen sind Dokument "Fragenkatalog Datamining" zu finden.**
- Antworten auf Fragen im Notebook, Diskussionen und Beschreibung der Ergebnisse sind optional (aber empfohlen) und werden nicht bewertet.

* [Übersicht Data Mining Praktikum](https://maucher.pages.mi.hdm-stuttgart.de/ai/page/dm/)


# Einführung

## Lernziele:

In diesem Versuch sollen Kenntnisse in folgenden Themen vermittelt werden:

* __Ähnlichkeit:__ Verfahren zur Bestimmung der Ähnlichkeit zwischen Personen (Kunden) und Elementen (Produkten)
* __Empfehlungssysteme__ Collaborative Filtering 
* __Collaborative Filtering:__ Nutzerbezogener Ansatz und elementbasierter Ansatz

Sämtliche Verfahren und Algorithmen werden in Python implementiert.

## Theorie zur Vorbereitung

### Recommender Systeme

Recommender Systeme werden im E-Commerce eingesetzt um Werbung in Form von kundenspezifischen Empfehlungen zu verteilen. Weitläufig bekannt sind die Amazon-Empfehlungen, die entweder per e-mail geschickt oder nach dem Log-In in der Web-Page angezeigt werden. Diese Empfehlungen werden in Abhängigkeit von den bisher vom jeweiligen Kunden gekauften bzw. bewerteten Produkten erstellt. In diesem Versuch werden die derzeit wohl am weitest verbreiteteten Verfahren für die Erzeugung kundenspezifischer Empfehlungen vorgestellt, darunter das elementweise Collaborative Filtering, welches z.B. auch von Amazon eingesetzt wird.     

Direkt-Marketing Methoden wie die kundenspezifische Erzeugung und Bereitstellung von Werbung erfordern detaillierte Kunden- und Warenkorbanalysen. Kunden mit ähnlichem Kaufverhalten werden in Kundengruppen zusammengefasst. Die Warenkorbanalyse untersucht u.a. welche Waren bevorzugt im Verbund von der gleichen Person gekauft werden. Damit kann ein Händler Werbung in Form von Empfehlungen individuell und gezielt an seine Kunden richten, abhängig davon welcher Kundengruppe er angehört und welche Produkte bevorzugt von dieser Kundengruppe nachgefragt werden. 

Im ersten Teil der Übung werden fiktive Daten in einer überschaubaren Menge verwendet. Es handelt sich hier um Filmbewertungen. Anhand dieses Beispiels sollen die notwendigen Methoden und Abläufe implementiert und getestet werden. Diese werden im zweiten Teil der Übung auf echte Daten angewandt. Hierzu werden über eine Python-API Daten vom Internet-Meta-Radio _last.fm_ integriert. Auf der Basis dieser Daten sollen dann Musikempfehlungen für last.fm User berechnet werden. 

Recommender Systeme lassen sich mit

* Clustering Verfahren
* Suchalgorithmen
* Collaborativen Filtering 
 
realisieren. Am häufigsten wird hierbei das Collaborative Filtering eingesetzt. Für das Collaborative Filtering wird jeder der $M$ User durch einen $N$-dimensionalen Vektor beschrieben, wobei $N$ die Anzahl der Produkte im Angebot des Händlers ist. Jedes Element im Vektor gehört zu einem speziellen Produkt. Das Element hat den Wert 1, wenn der User dieses Produkt bereits gekauft hat, sonst 0 (andere Wertbelegungen sind möglich, z.B. wenn Produktbewertungen vorliegen). Alle $M$ Zeilenvektoren können zur _User/Item_ Matrix zusammengefasst werden (siehe Abbildung).


<img src="https://maucher.home.hdm-stuttgart.de/Pics/UserItemMatrix.png" style="width:500px" align="center">

Das traditionelle **userbasierte Collaborative Filtering (UCF)**, benutzt die Ähnlichkeit zwischen Benutzern: Um für User $U_i$ eine Empfehlung zu erzeugen wird zunächst der diesem User ähnlichste Kunde (oder eine Menge vom ähnlichsten Kunden) ermittelt. Dann werden $U_i$ die Produkte (Items) empfohlen, welche der ähnlichste Kunde gekauft hat, $U_i$ selbst jedoch noch nicht. 

Dieser Ansatz skaliert schlecht im Fall sehr großer *User/Item*-Matrizen. Ausserdem ist er für User, welche erst wenige Produkte gekauft haben unzuverlässig. Besser eignet sich in diesen Fällen das **itembasierte Collaborative Filtering (ICF)**. Es wird u.a. von Amazon.com eingesetzt. Diese Variante benutzt die Ähnlichkeit zwischen Produkten (Items). Dabei sind Produkte umso ähnlicher je mehr Kunden diese Produkte gemeinsam gekauft haben. Für die Produkte welche ein Referenzuser $U_i$ bereits gekauft hat, werden die ähnlichsten Produkte ermittelt. Diese ähnlichsten Produkte werden $U_i$ empfohlen, wenn er sie nicht schon selbst gekauft hat.

Im folgenden Abschnitt werden einige gebräuchliche Metriken für die Berechnung der Ähnlichkeit zwischen Benutzern oder Artikeln vorgestellt. Für Collaboratives Filtering wird sehr häufig das Cosinus - Ähnlichkeitsmaß eingesetzt.


### Gebräuchliche Ähnlichkeitsmaße

Die __euklidische Distanz__ $d_E(\underline{a},\underline{b})$ zwischen zwei n-dimensionalen Vektoren $\underline{a}=(a_1,\ldots,a_n)$ und $\underline{b}=(b_1,\ldots,b_n)$ berechnet sich zu
	$$
	d_E(\underline{a},\underline{b})=\sqrt{\sum_{i=1}^n (a_i-b_i)^2}
	$$
Zwei Vektoren können als umso ähnlicher erachtet werden, je kleiner deren euklidische Distanz ist. 
Ein auf der euklidischen Metrik basierendes Ähnlichkeitsmaß zwischen zwei Vektoren $\underline{a}$ und $\underline{b}$ kann durch 
$$
s_E(\underline{a},\underline{b})=\frac{1}{1+d_E(\underline{a},\underline{b})}
$$
angegeben werden.


__Pearson Korrelation__
Die Ähnlichkeit zwischen zwei Vektoren kann auch durch den Pearson-Korrelationskoeffizient $\rho_{\underline{a},\underline{b}}$ ausgedrückt werden. Er berechnet sich zu
$$
\rho_{\underline{a},\underline{b}}= \frac{1}{N}\cdot \sum\limits_{i=1}^{N}\frac{(a_i-\overline{a})}{\sigma_a} \frac{(b_i-\overline{b})}{\sigma_b}
$$
Dabei bezeichnet $N$ die Länge der Vektoren, $\overline{a}$ den Mittelwert und $\sigma_a$ die Standardabweichung des Vektors $\underline{a}$. 

Der Pearson-Korrelationskoeffizient misst die lineare Abhängigkeit zwischen zwei Vektoren. Der maximale Wert von $+1$ wird erreicht, wenn die durch die beiden Vektoren definierten N Punkte im 2-dimensionalen Raum auf einer ansteigenden Geraden liegen. Der Minimalwert von $-1$ wird erreicht, wenn die Punkte auf einer abfallenden Geraden liegen. Der Betrag des Koeffizienten ist umso kleiner, je stärker die Punkte von einer fiktiven Geraden (kann durch lineare Regression berechnet werden) abweichen. Der Koeffizient ist $0$ wenn keine lineare Abhängigkeit zwischen den Vektoren besteht.


__Cosinus Ähnlichkeitsmaß__
Die Ähnlichkeit zwischen zwei Vektoren kann auch durch den Cosinus $\cos(\underline{a},\underline{b})$ ausgedrückt werden. Er berechnet sich zu
$$
\cos(\underline{a},\underline{b})= \frac{\underline{a} \cdot \underline{b}}{\left\|\underline{a}\right\|\cdot \left\|\underline{b}\right\|}
$$
wobei im Zähler das Skalarprodukt der beiden Vektoren steht und mit $\left\|\underline{x}\right\|$ der Betrag des Vektors $\underline{x}$ bezeichnet wird.

Falls die Vektoren $\underline{a}$ und $\underline{b}$ mittelwertfrei sind, ist der Cosinus-Ähnlichkeitswert gleich dem Pearson-Korrelationswert. In der Dokument- und Textanalyse wird vornehmlich das Cosinus-Ähnlichkeitsmaß verwendet. 


__Russel Rao Ähnlichkeitsmaß__
Die Russel Rao-Ähnlichkeit zwischen zwei binären Vektoren $\underline{a}$ und $\underline{b}$ mißt das Verhältnis zwischen der Anzahl $\alpha$ der Stellen in denen beide Vektoren den Wert 1 haben und der Länge $n$ der Vektoren. Z.B. ist für die Vektoren $\underline{a}=(1,0,1,0,0,1)$ und $\underline{b}=(0,1,1,1,0,1)$ die Russel-Rao-Ähnlichkeit $s_{RR}(\underline{a},\underline{b})=2/6=0.333$.

__Jaccard Ähnlichkeitsmaß__
Die Jaccard-Ähnlichkeit zwischen zwei binären Vektoren $\underline{a}$ und $\underline{b}$ mißt das Verhältnis zwischen der Anzahl $\alpha$ der Stellen in denen beide Vektoren den Wert $1$ haben und der Anzahl der Stellen in denen mindestens einer der beiden Vektoren ungleich $0$ ist. Z.B. ist für die Vektoren $\underline{a}=(1,0,1,0,0,1)$ und $\underline{b}=(0,1,1,1,0,1)$ die Jaccard-Ähnlichkeit $s_{J}(\underline{a},\underline{b})=2/5=0.4$. Die Jaccard Metrik wird in diesem Versuch für die Bestimmung der Ähnlichkeit von *last.fm*-Usern eingesetzt.


## Vor dem Versuch zu klärende Fragen
Eine Untermenge der im Folgenden aufgeführten Fragen wird zu Beginn des Versuchs im Rahmen eines Gruppenkolloqs abgefragt. Auf jede Frage sollte von mindestens einem Gruppenmitglied eine Antwort geliefert werden und jedes Gruppenmitglied muss mindestens eine der gestellten Fragen beantworten können.

**Aufgaben:**

* Beschreiben Sie das Prinzip des userbasierten Collaborativen Filtering (UCF).

Antwort: Beim UCF wird für einen User der ihm ähnlichste User (in Bezug auf gekaufte Produkte) rausgesucht und Produkte die dieser ähnliche User schon gekauft hat, der betrachtete User aber nicht, vorgeschlagen.

* Welche Nachteile hat das UCF?

Antwort: Es skaliert schlecht auf sehr große Usermengen. Außerdem ist es für User die erst wenige Produkte gekauft haben unzuverlässig.

* Worin besteht der Unterschied zwischen UCF und itembasierten Collaborativen Filtering (ICF)?

Antwort: Beim itembasierten Collaborativen Filtering werden, anders als beim UCF, nicht ähnliche User miteinander verglichen, sondern Produkte. Produkte sind sich je ähnlicher, desto mehr User diese gemeinsam gekauft haben.

* Gegeben seien die Vektoren 

    \begin{eqnarray*}
    \underline{a} & = & [1,2,3,4,5,6] \\
    \underline{b} & = & [3,3,5,6,7,8] \\
    \end{eqnarray*}
    
    Schreiben Sie eine Python Funktion, die den Mittelwert derartiger Vektoren berechnet. Schreiben Sie eine weitere Funktion, die die Varianz berechnet

In [1]:
import numpy as np

def mean_vec(vec):
    _sum=0
    for i in vec:
        _sum+=i
    return _sum/len(vec)

def variance_vec(vec):
    _sum=0
    for i in range(len(vec)):
        _sum += (vec[i]-mean_vec(vec))**2
    return _sum/len(vec)

In [2]:
a = [1,2,3,4,5,6]
b = [3,3,5,6,7,8]

print(f"a: Mittelwert: {mean_vec(a)} ,  Varianz: {variance_vec(a)}")
print(f"b: Mittelwert: {mean_vec(b)} ,  Varianz: {variance_vec(b)}")

a: Mittelwert: 3.5 ,  Varianz: 2.9166666666666665
b: Mittelwert: 5.333333333333333 ,  Varianz: 3.5555555555555554


* Wie groß ist die

    - Euklidische Ähnlichkeit
    - Pearson Ähnlichkeit
    - Cosinus Ähnlichkeit
    
    zwischen den Vektoren $\underline{a}$ und $\underline{b}$? 

In [3]:
def euk_similarity(a,b):
    _sum=0
    for i in range(len(a)):
        _sum += (a[i]-b[i])**2
    return 1/(1+_sum** (1/2))

def pear_similarity(a,b):
    _sum=0
    for i in range(len(a)):
        _sum += ((a[i]- mean_vec(a)) / np.sqrt(variance_vec(a) )) * ((b[i]- mean_vec(b)) / np.sqrt(variance_vec(b) ))
        
    return _sum/len(a)

def cosine_similarity(a,b):
    prod=0
    sqsuma= 0
    sqsumb= 0
    for i in range(len(a)):
        prod += a[i]*b[i]
        
        sqsuma += a[i]**2
        sqsumb += b[i]**2
        
    return prod / (np.sqrt(sqsuma) * np.sqrt(sqsumb))

In [4]:
print(f"Euklidische Ähnlichkeit: {euk_similarity(a,b)}")
print(f"Pearson Ähnlichkeit: {pear_similarity(a,b)}")
print(f"Cosinus Ähnlichkeit: {cosine_similarity(a,b)}")

Euklidische Ähnlichkeit: 0.179128784747792
Pearson Ähnlichkeit: 0.9833434220628549
Cosinus Ähnlichkeit: 0.991060084745164


* In welchen Fällen sind Cosinus- und Pearsonähnlichkeit der euklidischen Ähnlichkeit vorzuziehen?

Antwort: Wenn die zu vergleichenden Elemente verschieden groß sind. Zum Beispiel wenn zwei Texte eigentlich sehr ähnlich sind, aber unterschiedlich lang, würde die euklidische Ähnlichkeit allein anhand der unterschiedlichen Länge einen großen Unterschied zeigen.

In [5]:
from IPython.display import Latex
from IPython.display import Image
import pylast

# Versuchsdurchführung
## Teil 1: Fiktive Filmbewertung
### Daten
Folgende Tabelle enthält die Filmbewertungen von 7 Personen.
from IPython.display import Latex
In diesem Versuch sollen Kenntnisse in folgenden Themen vermittelt werden:


<img src="https://maucher.home.hdm-stuttgart.de/Pics/recommenderFilmRecommendations.PNG" style="width:500px" align="center">



Die Tabelle ist als Python dictionary _critics_ implementiert. Die Keys des Python-Dictionary definieren die Namen von Personen (Zeilen in der Matrix), die Filme bewertet haben. Die Values sind selbst wieder Dictionarys, welche als Keys die Filmnamen (Spalten in der Matrix) und als Values die jeweilige Filmbewertung (Matrixelment) enthalten.

In [6]:
critics={'Lisa Rose': {'Lady in the Water': 2.5, 'Snakes on a Plane': 3.5,
 'Just My Luck': 3.0, 'Superman Returns': 3.5, 'You, Me and Dupree': 2.5, 
 'The Night Listener': 3.0},
'Gene Seymour': {'Lady in the Water': 3.0, 'Snakes on a Plane': 3.5, 
 'Just My Luck': 1.5, 'Superman Returns': 5.0, 'The Night Listener': 3.0, 
 'You, Me and Dupree': 3.5}, 
'Michael Phillips': {'Lady in the Water': 2.5, 'Snakes on a Plane': 3.0,
 'Superman Returns': 3.5, 'The Night Listener': 4.0},
'Claudia Puig': {'Snakes on a Plane': 3.5, 'Just My Luck': 3.0,
 'The Night Listener': 4.5, 'Superman Returns': 4.0, 
 'You, Me and Dupree': 2.5},
'Mick LaSalle': {'Lady in the Water': 3.0, 'Snakes on a Plane': 4.0, 
 'Just My Luck': 2.0, 'Superman Returns': 3.0, 'The Night Listener': 3.0,
 'You, Me and Dupree': 2.0}, 
'Jack Matthews': {'Lady in the Water': 3.0, 'Snakes on a Plane': 4.0,
 'The Night Listener': 3.0, 'Superman Returns': 5.0, 'You, Me and Dupree': 3.5},
'Toby': {'Snakes on a Plane':4.5,'You, Me and Dupree':1.0,'Superman Returns':4.0}
}

### Ähnlichkeiten berechnen

Für die Bestimmung der Ähnlichkeit zwischen Personen und Produkten werden in diesem Versuch ein auf der euklidischen Distanz basierendes Ähnlichkeitsmaß und die Pearson-Korrelation verwendet. Beide Ähnlichkeitsmaße sind in den unten definierten Funktionen implementiert. Alle drei hier implementierten Funktionen zur Berechnung der Ähnlichkeit erhalten als Übergabeparameter das oben definierte Dictionary, das die Filmbewertungen enthält und die Namen der zwei Personen, die verglichen werden sollen. 

Zu beachten ist, dass in beiden Funktionen für die Berechnung der Ähnlichkeit zwischen zwei Personen nur die Produkte berücksichtigt werden, welche von beiden Personen schon bewertet wurden. Es handelt sich hier also um modifizierte Ähnlichkeitsfunktionen. 

__Aufgabe:__
Fragen Sie von diesem Dictionary _Toby's_ Bewertung des Films _Snakes on a Plane_ ab und geben Sie diesen Wert aus: 

In [7]:
print(critics.get("Toby").get("Snakes on a Plane"))

4.5


In [8]:
import numpy as np
import scipy.spatial.distance as sci


def sim_euclid(prefs,person1,person2,normed=True):
    ''' Returns a euclidean-distance-based similarity score for 
    person1 and person2. In the distance calculation the sum is computed 
    only over those items, which are nonzero for both instances, i.e. only
    films which are ranked by both persons are regarded.
    If the parameter normed is True, then the euclidean distance is divided by
    the number of non-zero elements integrated in the distance calculation. Thus
    the effect of larger distances in the case of an increasing number of commonly ranked
    items is avoided.
    '''
    # Get the list of shared_items
    si={}
    for item in prefs[person1]: 
        if item in prefs[person2]: 
            si[item] = 1
            
    # len(si) counts the number of common ratings
    # if they have no ratings in common, return 0
    if len(si) == 0: 
        return 0

    # Add up the squares of all the differences
    sum_of_squares=np.sqrt(sum([pow(prefs[person1][item]-prefs[person2][item],2) 
                     for item in prefs[person1] if item in prefs[person2]]))
    if normed:
        sum_of_squares= 1.0/len(si)*sum_of_squares
    return 1/(1+sum_of_squares)


def sim_pearson(prefs,p1,p2):
    '''
    Returns the Pearson correlation coefficient for p1 and p2
    '''

    # Get the list of commonly rated items
    si={}
    for item in prefs[p1]: 
        if item in prefs[p2]: 
            si[item]=1

    # if they are no ratings in common, return 0
    if len(si)==0: return 0

    # Sum calculations
    n=len(si)

    # Calculate means of person 1 and 2
    mp1=np.mean([prefs[p1][it] for it in si])
    mp2=np.mean([prefs[p2][it] for it in si])

    # Calculate standard deviation of person 1 and 2
    sp1=np.std([prefs[p1][it] for it in si])
    sp2=np.std([prefs[p2][it] for it in si])

    # If all elements in one sample are identical, the standard deviation is 0. 
    # In this case there is no linear correlation between the samples
    if sp1==0 or sp2==0:
        return 0
    r=1/(n*sp1*sp2)*sum([(prefs[p1][it]-mp1)*(prefs[p2][it]-mp2) for it in si])
    return r


def sim_RusselRao(prefs,person1,person2,normed=True):
    ''' Returns RusselRao similaritiy between 2 users. The RusselRao similarity just counts the number
    of common non-zero components of the two vectors and divides this number by N, where N is the length
    of the vectors. If normed=False, the division by N is omitted.
    '''
    # Get the list of shared_items
    si={}
    commons=0
    for item in prefs[person1]: 
        if prefs[person1][item]==1 and prefs[person2][item]==1:   
            commons+=1
    #print commons
    if not normed:
        return commons
    else:
        return commons*1.0/len(prefs[person1]) 

**Aufgabe:**
1. Geben Sie die euklidische Ähnlichkeit und die Pearson Ähnlichkeit zwischen den Personen _Toby_ und _Lisa Rose_ aus.
2. Diskutieren Sie die unterschiedlichen Ähnlichkeitswert.

1:

In [9]:
# 1.
print("Euklidische Ähnlichkeit:", sim_euclid(critics, "Toby", "Lisa Rose"))
print("Pearson Ähnlichkeit:", sim_pearson(critics, "Toby", "Lisa Rose"))

Euklidische Ähnlichkeit: 0.615911621788925
Pearson Ähnlichkeit: 0.9912407071619302


__Aufgabe:__
0. Schreiben Sie eine Funktion `topMatches(prefs,person,similarity)`, welche für eine beliebige in _critics_ enthaltene Person die Ähnlichkeitswerte zu allen anderen Personen berechnet und in einer geordneten Liste zurück gibt. Der Funktion soll als Übergabeparameter auch die anzuwendende Ähnlichkeitsfunktion (`sim_euclid` oder `sim_pearson`) übergeben werden können. Berechnen Sie mit dieser Funktion für jede Person die *top matches*, zunächst unter Verwendung der euklidischen- dann unter Verwendung der Pearson-Ähnlichkeit.
1. Geben Sie mit der implementierten Funktion die *top matches* der Person Toby aus.
2. Vergleichen Sie die beiden Ähnlichkeitsmaße. Welches Ähnlichkeitsmaß erscheint Ihnen für diesen Anwendungsfall sinnvoller und warum?

0:

In [10]:
def topMatches(prefs,person,similarity): #f
    aehn_dict = []
    for pers in prefs:
        if (pers != person):
            aehn_dict.append((pers, similarity(prefs,person,pers)))
    sorted_dict = sorted(aehn_dict, key=lambda x: x[1], reverse=True)
    return sorted_dict

In [11]:
testList = list(critics.keys())
print("Top Matches Euklid:")
for p in testList:
    print(p, ":", topMatches(critics, p, sim_euclid)[0][0])
    
print("\nTop Matches Pearson:")
for p in testList:
    print(p, ":", topMatches(critics, p, sim_pearson)[0][0])

Top Matches Euklid:
Lisa Rose : Mick LaSalle
Gene Seymour : Jack Matthews
Michael Phillips : Lisa Rose
Claudia Puig : Michael Phillips
Mick LaSalle : Lisa Rose
Jack Matthews : Gene Seymour
Toby : Mick LaSalle

Top Matches Pearson:
Lisa Rose : Toby
Gene Seymour : Jack Matthews
Michael Phillips : Claudia Puig
Claudia Puig : Michael Phillips
Mick LaSalle : Toby
Jack Matthews : Gene Seymour
Toby : Lisa Rose


1:

In [12]:
print("Top Matches Toby Euklid:")
display(topMatches(critics,"Toby",sim_euclid)) 
print("Top Matches Toby Pearson:")
display(topMatches(critics,"Toby",sim_pearson))

Top Matches Toby Euklid:


[('Mick LaSalle', 0.6666666666666666),
 ('Claudia Puig', 0.6246387977050463),
 ('Lisa Rose', 0.615911621788925),
 ('Michael Phillips', 0.5584815598877471),
 ('Jack Matthews', 0.5227744249483389),
 ('Gene Seymour', 0.5108747069239427)]

Top Matches Toby Pearson:


[('Lisa Rose', 0.9912407071619302),
 ('Mick LaSalle', 0.9244734516419051),
 ('Claudia Puig', 0.8934051474415642),
 ('Jack Matthews', 0.6628489803598703),
 ('Gene Seymour', 0.3812464258315117),
 ('Michael Phillips', -1.0)]

### Berechnung von Empfehlungen mit User basiertem Collaborative Filtering
Für die Produkte, die von einer Person noch nicht gekauft wurden, sollen Empfehlungen berechnet werden. Die Empfehlungen können ebenfalls Werte zwischen 1 (wird nicht empfohlen) und 5 (wird stark empfohlen) annehmen. Für die Berechnung der Empfehlung werden die Bewertungen des jeweiligen Produkts durch die anderen Personen herangezogen. Dabei werden die Bewertungen der ähnlichen Personen (d.h. hoher Pearson-Korrelationswert) stärker mit einbezogen als die Bewertungen durch Personen mit einem niedrigen Korrelationswert.

__Beispiel:__
Toby hat die Filme *The Night Listener*, _Lady in the Water_ und _Just My Luck_ noch nicht gekauft. Für diese Filme soll für Toby eine Empfehlung berechnet werden.
In der unten aufgeführten Tabelle enthält die zweite Spalte die _Pearson-Ähnlichkeitswerte_ zwischen Toby und den anderen Personen. Die Spalten 3, 5 und 7 enthalten die Bewertungen der Filme *The Night Listener*, _Lady in the Water_ und _Just My Luck_ durch die anderen Personen. Die Spalten 4, 6 und 8 enthalten die jeweilige Filmbewertung gewichtet (mulipliziert) mit den Ähnlichkeitswerten der jeweiligen Person. Es fällt auf, dass in der Tabelle _Michael_ nicht enthalten ist. Das liegt daran, dass _Michael_ und _Toby_ einen negativen Ähnlichkeitswert aufweisen, d.h. deren Interessen sind gegenläufig. Personen mit negativem Ähnlichkeitswert sollten für Empfehlungen nicht berücksichtigt werden.
Die Zeile _Sum_ enthält die Summe aller gewichteten Bewertungen. Aus diesem Wert allein kann die Empfehlung noch nicht abgeleitet werden, da Filme die nur von wenigen Personen bewertet wurden, eine relativ kleine Summe ergeben. Deshalb sollte _Sum_ noch durch die Anzahl der Bewertungen für diesen Film geteilt werden. Oder besser: Nicht durch die Summe der Bewertungen, sondern durch die Summe der relevanten Ähnlichkeitswerte (*KSum*). Der resultierende Empfehlungswert ist in der letzten Zeile eingetragen.



<img src="https://maucher.home.hdm-stuttgart.de/Pics/recommenderFilmCalculation.PNG" style="width:500px" align="center">




__Aufgabe:__
Schreiben Sie eine Funktion *getRecommendations(prefs,person,similarity)*, mit der die Empfehlungswerte berechnet werden können und bestimmen Sie die Empfehlungswerte für Toby. Der Funktion wird  

* das Dictionary _critics_ mit den Filmbewertungen, 
* der Name der Person, für welche Empfehlungen berechnet werden sollen
* die Methode für die Berechnung der Ähnlichkeit *sim_euclid* oder *sim_pearson*

übergeben. Die Methode soll eine geordnete Liste zurück geben. Jedes Listenelement enthält an erster Stelle den berechneten Empfehlungswert und an zweiter Stelle den Namen des Films. Die Liste soll nach Empfehlungswerten absteigend geordnet sein.

Testen Sie diese Funktion indem Sie die Empfehlungen für _Toby_ berechnen und mit den Werten in der oben aufgeführten Tabelle vergleichen.

In [13]:
def getRecommendations(prefs,person,similarity):
    unseen = []
    for pref in prefs:
        for movie in prefs.get(pref):
            if movie not in unseen:
                unseen.append(movie)         
    for seen_film in prefs.get(person):
        if prefs.get(person).get(seen_film) != 0:
            if (seen_film in unseen):
                unseen.remove(seen_film)
    recom_list = []
    for movie in unseen:
        movie_scores = []
        sim_scores = []
        for other_person in prefs:
            if other_person != person:
                s_score = similarity(prefs,person,other_person)
                if ((s_score >= 0) & (movie in prefs.get(other_person))) :
                    m_score = prefs.get(other_person).get(movie)
                    sim_scores.append(s_score)
                    movie_scores.append(s_score * m_score)         
        movie_sum = np.sum(movie_scores)
        score_sum = np.sum(sim_scores)
        recom_list.append(((movie_sum/score_sum), movie ))
    return sorted(recom_list,key=lambda x: x[0], reverse=True)

In [14]:
getRecommendations(critics,"Toby",sim_pearson)

[(3.3477895267131004, 'The Night Listener'),
 (2.832549918264161, 'Lady in the Water'),
 (2.530980703765564, 'Just My Luck')]

### Berechnung von Empfehlungen mit Item basiertem Collaborative Filtering
In den vorigen Aufgaben wurden Ähnlichkeiten zwischen Personen bestimmt und für Produktempfehlungen benutzt (User basiertes Collaborative Filtering). Jetzt soll die Ähnlichkeit zwischen Produkten berechnet werden und auf der Basis dieser Produktähnlichkeit Empfehlungen berechnet werden (Item basiertes Collaborative Filtering).

Dabei sollen die bereits implementierten Ähnlichkeitsfunktion *sim_euclid* und *sim_pearson* sowie die Ähnlichkeeits-Sortierfunktion *topMatches* unverändert eingesetzt werden.

__Aufgabe:__

1. Implementieren Sie eine Funktion, welche das Bewertungsdictionary *critics* derart transformiert, dass die Funktionen `sim_euclid`, `sim_pearson` und `topMatches` für das Item-basierte CF unverändert eingesetzt werden können. Die transformierte Matrix soll unter dem Namen *transCritics* abgespeichert werden.
2. Schreiben Sie eine Funktion `calculateSimilarItems`, die aus der transformierten Matrix *transCritics* ein Dictionary berechnet, welches die Ähnlichkeit zwischen allen Filmen beschreibt. Die Keys des Dictionary sind die Filmnamen. Die Values sind geordnete Listen, welche die Funktion `topMatches` zurückgibt, wenn sie für die Filme (nicht für die User) aufgerufen wird. Dieses Dictionary wird an das aufrufende Programm zurück geben. 
3. Schreiben Sie eine Funktion `getRecommendedItems`, welche basierend auf dem im unten aufgeführten Beispiel dargestellten Verfahren unter Vorgabe der Bewertungsmatrix und der zu verwendenden Ähnlichkeitsfunktion Produktempfehlungen berechnet.
4. Testen Sie die Funktion indem Sie die Empfehlungen für Toby berechnen und mit den Werten in der unten aufgeführten Tabelle vergleichen

__Erläuterndes Beispiel:__

_Toby_ hat die Filme *The Night Listener*, *Lady in the Water* und *Just My Luck* noch nicht gekauft. Für diese Filme soll für *Toby* eine Empfehlung berechnet werden. Gekauft und bewertet hat *Toby* die Filme *Snakes on a plane*, *Superman Returns* und *You and me and Dupree*. Diese bereits vorhandenen Filme bilden die erste Spalte der unten dargestellten Matrix. In der zweiten Spalte befinden sich _Toby's_ Bewertungen dieser Filme. Die Spalten 3,5 und 7 enthalten die Ähnlichkeitswerte (mit *calculateSimilarItems* unter Verwendung des normierten euklidischen Ähnlichkeitsmaßes berechnet) zwischen den drei von *Toby* noch nicht gekauften Filmen und den drei von _Toby_ bewerteten Filmen. Diese Ähnlichkeitswerte werden jeweils mit _Toby's_ Bewertungen multipliziert. Das Resultat dieser Multiplikation befindet sich in den Spalten 4,6 und 8. Der finale Empfehlungswert für die von _Toby_ noch nicht gekauften Filme wird berechnet in dem in den Spalten 4,6 und 8 zunächst die Summe über die Werte dieser Spalte in den drei oberen Zeilen berechnet wird und durch die Summe über die Werte der Spalten 3,5 und 7 geteilt wird. Im Fall, dass die *Pearson-Korrelation* zwischen den Filmen als Ähnlichkeitswert herangezogen wird, können negative Ähnlichkeitswerte auftreten. Dann soll in die Berechnung eines Empfehlungswert für Film A nur dann die Bewertung von Film B einfließen, wenn der Korrelationswert zwischen beiden $>0$ ist.  


<img src="https://maucher.home.hdm-stuttgart.de/Pics/recommenderFilmItemBased.PNG" style="width:500px" align="center">


1:

In [15]:
def transformCritics(prefs):
    movies = []
    transCritics = {}
    for person in prefs:
        for movie in prefs.get(person):
            if movie not in movies:
                movies.append(movie)
    for movie in movies:
        temp_dict = {}
        for person in prefs:
            
            score = prefs.get(person).get(movie)
            if(score != None):
                temp_dict.update({person: score})
        transCritics.update({movie: temp_dict})
        
    
    return transCritics

In [16]:
transCritics = transformCritics(critics)
transCritics

{'Lady in the Water': {'Lisa Rose': 2.5,
  'Gene Seymour': 3.0,
  'Michael Phillips': 2.5,
  'Mick LaSalle': 3.0,
  'Jack Matthews': 3.0},
 'Snakes on a Plane': {'Lisa Rose': 3.5,
  'Gene Seymour': 3.5,
  'Michael Phillips': 3.0,
  'Claudia Puig': 3.5,
  'Mick LaSalle': 4.0,
  'Jack Matthews': 4.0,
  'Toby': 4.5},
 'Just My Luck': {'Lisa Rose': 3.0,
  'Gene Seymour': 1.5,
  'Claudia Puig': 3.0,
  'Mick LaSalle': 2.0},
 'Superman Returns': {'Lisa Rose': 3.5,
  'Gene Seymour': 5.0,
  'Michael Phillips': 3.5,
  'Claudia Puig': 4.0,
  'Mick LaSalle': 3.0,
  'Jack Matthews': 5.0,
  'Toby': 4.0},
 'You, Me and Dupree': {'Lisa Rose': 2.5,
  'Gene Seymour': 3.5,
  'Claudia Puig': 2.5,
  'Mick LaSalle': 2.0,
  'Jack Matthews': 3.5,
  'Toby': 1.0},
 'The Night Listener': {'Lisa Rose': 3.0,
  'Gene Seymour': 3.0,
  'Michael Phillips': 4.0,
  'Claudia Puig': 4.5,
  'Mick LaSalle': 3.0,
  'Jack Matthews': 3.0}}

2:

In [17]:
def calculateSimilarItems(prefs,similarity):
    sim_dict={}
    for movie in prefs:
        matches = topMatches(prefs,movie,similarity)
        sim_dict[movie] = matches
    return sim_dict

In [18]:
calculateSimilarItems(transCritics,sim_euclid)

{'Lady in the Water': [('You, Me and Dupree', 0.7655876216850789),
  ('The Night Listener', 0.7597469266479578),
  ('Snakes on a Plane', 0.7277142573518672),
  ('Just My Luck', 0.615911621788925),
  ('Superman Returns', 0.6125741132772068)],
 'Snakes on a Plane': [('Superman Returns', 0.7578982763068516),
  ('The Night Listener', 0.7387961250362586),
  ('Lady in the Water', 0.7277142573518672),
  ('You, Me and Dupree', 0.5824585256498355),
  ('Just My Luck', 0.5784128280412532)],
 'Just My Luck': [('You, Me and Dupree', 0.6534537935444722),
  ('The Night Listener', 0.6303969981288705),
  ('Lady in the Water', 0.615911621788925),
  ('Snakes on a Plane', 0.5784128280412532),
  ('Superman Returns', 0.5123025255147889)],
 'Superman Returns': [('Snakes on a Plane', 0.7578982763068516),
  ('The Night Listener', 0.6697893816771064),
  ('Lady in the Water', 0.6125741132772068),
  ('You, Me and Dupree', 0.5874822290669),
  ('Just My Luck', 0.5123025255147889)],
 'You, Me and Dupree': [('Lady in

3:

In [19]:
def getRecommendedItems(prefs,person,similarity):
    recom_dict = {}
    unseen = []
    seen_score = []
    for person in prefs:
        for movie in prefs.get(person):
            if movie not in unseen:
                unseen.append(movie)
    for seen,score in prefs.get(person).items():
        if (seen in unseen):
            unseen.remove(seen)
            seen_score.append((seen,score))
            
    sim_dict = calculateSimilarItems(transformCritics(prefs),similarity)

    for unseen_movie in unseen:
        unseen_sim_sum = 0
        unseen_prod_sum = 0
        for movie,score in seen_score:
            movs = sim_dict.get(unseen_movie)
            for mov in movs:
                if (mov[0] == movie):
                    sim_score = mov[1]       
            if sim_score >= 0:
                unseen_sim_sum += sim_score
                prod_score = sim_score * score
                unseen_prod_sum += prod_score
            
            if unseen_sim_sum != 0 :
                recom_score = unseen_prod_sum/unseen_sim_sum
            else:
                recom_score = 0
        recom_dict.update({unseen_movie: recom_score})
        
    return recom_dict

4:

In [20]:
getRecommendedItems(critics,"Toby",sim_euclid) # Werte stimmen mit Tabelle überein

{'Lady in the Water': 3.082136961799338,
 'Just My Luck': 3.041861869079099,
 'The Night Listener': 3.20449096016088}

## last.fm Musikempfehlungen

__Aufgabe:__

1. Installieren Sie das Package pylast. Stellen Sie durch Aufruf der Funktion *network=pylast.LastFMNetwork()* eine Verbindung zu *last.fm* her. Beim Aufruf der Funktion wird die Anmeldung und Authentifizierung durchgeführt. Die Funktion gibt ein Objekt der Klasse *Network* zurück. Über dieses Objekt können Methoden, wie

    * *get_artist("kuenstlerName")* (liefert Objekt der Klasse *Artist*)
    * *get_album("albumName")* (liefert Objekt der Klasse *Album*)
    * *get_track("songName")* (liefert Objekt der Klasse *Track*)
    * *get_user("userName"):* (liefert Objekt der Klasse *Tag*)
    * usw.
    
      aufgerufen werden. Die Menge aller verfügbaren Klassen und deren Attribute und Methoden können dem Modul _pylast.py_ entnommen werden. In der untenstehenden Codel Zelle können Sie beispielsweise über das *Network*-Objekt die Methode `get_artist("BandIhrerWahl")` aufrufen um die von lastFM berechneten ähnlichsten Bands anzuzeigen.

2. Führen Sie die untenstehende Code Zelle aus um eine vordefinierte Gruppe von Benutzern, passend zu Band 'Slipknot' zu bekommen. Falls Sie ein lastfm Konto haben können Sie manuell auch andere Bands und Benutzer eintragen.
3. Implementieren Sie eine Funktion `createLastfmUserDict()`. Dieser Funktion soll, die oben angelegte Liste von *User*-Objekten _group_ übergeben werden. Für jeden User in *group* sollen die 20 beliebtesten Bands mit der Methode `topartists=*User*.get_top_artists()[0:20]` bestimmt werden. Die Methode gibt eine Liste von *Artist*-Objekt/Gewichtung-Paaren zurück. Die Gewichtungen von Objekten werden in diesem Versuch nicht benötigt. Auf das *i.te* *Artist*-Objekt selbst kann mit `topartists[i].item` zugegriffen werden. Die Menge aller Bands, die auf diese Weise gesammelt werden, wird im folgenden mit _AllBands_ bezeichnet. D.h. in *AllBands* befinden sich alle Bands, die für mindestens einen User in *group* zu den Top-20 gehören. Nun soll ein verschachteltes Dictionary mit Namen *userDict* wie folgt angelegt werden:

    * Die Keys sind die Namen der _User_-Objekte in _group_. Auf den Namen eines Objekts kann mit `get_name()` zugegriffen werden.
    * Die Values sind selbst wieder Dictionaries, deren Keys die Namen der Bands in *AllBands* sind. Achten Sie auch hier darauf, dass Sie nicht das *Artist*-Objekt selbst, sondern dessen Namen als Key verwenden. 
    * Für den User *a* und die Band *b* ist der Value `userDict[a][b]= 1`, falls *b* zu den Top-20 des Users *a* gehört. Andernfalls ist `userDict[a][b]= 0`. 
    
    Das derart angelegte Dictionary soll von der Funktion zurückgegeben werden. 
4. Wählen Sie jetzt einen beliebigen User aus *group*. Bestimmen Sie zu diesem User die ähnlichsten User in *group* durch Anwendung der im ersten Teilversuch implementierten Funktion `topMatches()`. Der Funktion wird das angelegte *userDict* und der Name des gewählten Users übergeben. Als Ähnlichkeitsmaß soll die euklidische Metrik angewandt werden.
5. Bestimmen Sie dann für den gewählten User Band-Empfehlungen durch Anwendung der im ersten Teilversuch implementierten Funktion `getRecommendations()`. Der Funktion wird das angelegte *userDict* und der Name des gewählten Users übergeben. Als Ähnlichkeitsmaß soll die euklidische Metrik, danach die Russel_Rao Metrik, angewandt werden.     
6. Diskutieren Sie das Ergebnis

1: und 2:

In [21]:
import pylast
nw=pylast.LastFMNetwork(api_key = "993a5bd9d79a98a53677570368d55acd",api_secret = "9b8de0b57903ac007cdd8ec9003b341e",username = "pythonlab")
band='Slipknot'
art1 = nw.get_artist(band)
print("Most similar (as calculated by lastFM) for artist: ",band)
for it in art1.get_similar(5):
    print("%3.3f \t %s"%(it.match, it.item))

print("\nApply predefined group of users")
usernames=['BrunoJoS','DPREBOYE','MPistol40','NemoNightfall','SkyRif','Wags1382','Znapsen','cortapsyco','emill_67','sattuviitana']
group=[]
for u in usernames:
    u1 = nw.get_user(u)
    group.append(u1)
print(group)

Most similar (as calculated by lastFM) for artist:  Slipknot
1.000 	 Korn
0.765 	 Stone Sour
0.632 	 System of a Down
0.598 	 Mudvayne
0.539 	 Vended

Apply predefined group of users
[pylast.User('BrunoJoS', pylast.LastFMNetwork('993a5bd9d79a98a53677570368d55acd', '9b8de0b57903ac007cdd8ec9003b341e', '', 'pythonlab', '')), pylast.User('DPREBOYE', pylast.LastFMNetwork('993a5bd9d79a98a53677570368d55acd', '9b8de0b57903ac007cdd8ec9003b341e', '', 'pythonlab', '')), pylast.User('MPistol40', pylast.LastFMNetwork('993a5bd9d79a98a53677570368d55acd', '9b8de0b57903ac007cdd8ec9003b341e', '', 'pythonlab', '')), pylast.User('NemoNightfall', pylast.LastFMNetwork('993a5bd9d79a98a53677570368d55acd', '9b8de0b57903ac007cdd8ec9003b341e', '', 'pythonlab', '')), pylast.User('SkyRif', pylast.LastFMNetwork('993a5bd9d79a98a53677570368d55acd', '9b8de0b57903ac007cdd8ec9003b341e', '', 'pythonlab', '')), pylast.User('Wags1382', pylast.LastFMNetwork('993a5bd9d79a98a53677570368d55acd', '9b8de0b57903ac007cdd8ec9003b34

3:

In [22]:
def createLastfmUserDict(group = group):
    allBands = []
    userDict = {}
    for user in group:
        topartists = user.get_top_artists()[0:20]
        userDict[user.get_name()] = {}
        for artist in topartists:
            userDict[user.get_name()].update({artist.item.get_name(): 1})
            if artist.item.get_name() not in allBands:
                allBands.append(artist.item.get_name())
    for band in allBands:
        for key in userDict:
            if band not in userDict[key]:
                userDict[key].update({band: 0})
    for key in userDict:
        userDict[key] = dict(sorted(userDict[key].items()))
    return userDict

In [23]:
userDict = createLastfmUserDict(group)

4:

In [24]:
topMatches(userDict, 'BrunoJoS', sim_euclid)

[('NemoNightfall', 0.9659253562958626),
 ('SkyRif', 0.9636581744069851),
 ('emill_67', 0.9625815550508859),
 ('sattuviitana', 0.9625815550508859),
 ('DPREBOYE', 0.9615384615384615),
 ('MPistol40', 0.9615384615384615),
 ('Znapsen', 0.9615384615384615),
 ('cortapsyco', 0.9615384615384615),
 ('Wags1382', 0.9605261285528738)]

5:

In [25]:
getRecommendations(userDict, 'BrunoJoS', sim_euclid)

[(0.5555549242855998, 'Korn'),
 (0.44480281354997675, 'Five Finger Death Punch'),
 (0.44442062761823037, 'Rage Against the Machine'),
 (0.33379281734763366, 'Stone Sour'),
 (0.33340676070899766, 'Metallica'),
 (0.33340676070899766, 'Red Hot Chili Peppers'),
 (0.2226546499733747, 'Trivium'),
 (0.22239289379976498, 'Sabaton'),
 (0.22227246404162834, 'Lamb of God'),
 (0.22214816357660205, 'Avenged Sevenfold'),
 (0.22214816357660205, 'In Flames'),
 (0.22214816357660205, 'Pink Floyd'),
 (0.22202773381846538, 'Limp Bizkit'),
 (0.22202773381846538, 'Pendulum'),
 (0.2219108554991623, 'Coheed and Cambria'),
 (0.11152035330600535, 'Alexisonfire'),
 (0.11152035330600535, 'All Time Low'),
 (0.11152035330600535, 'Ben Howard'),
 (0.11152035330600535, 'Billy Talent'),
 (0.11152035330600535, 'Breaking Benjamin'),
 (0.11152035330600535, 'Bring Me the Horizon'),
 (0.11152035330600535, 'Bullet for My Valentine'),
 (0.11152035330600535, 'Memphis May Fire'),
 (0.11152035330600535, 'Mumford & Sons'),
 (0.11

In [26]:
getRecommendations(userDict, 'BrunoJoS', sim_RusselRao)

[(0.56, 'Five Finger Death Punch'),
 (0.56, 'Korn'),
 (0.48000000000000004, 'Stone Sour'),
 (0.44, 'Rage Against the Machine'),
 (0.36, 'Metallica'),
 (0.36, 'Red Hot Chili Peppers'),
 (0.36, 'Trivium'),
 (0.28, 'Sabaton'),
 (0.24000000000000002, 'Alexisonfire'),
 (0.24000000000000002, 'All Time Low'),
 (0.24000000000000002, 'Ben Howard'),
 (0.24000000000000002, 'Billy Talent'),
 (0.24000000000000002, 'Breaking Benjamin'),
 (0.24000000000000002, 'Bring Me the Horizon'),
 (0.24000000000000002, 'Bullet for My Valentine'),
 (0.24000000000000002, 'Lamb of God'),
 (0.24000000000000002, 'Memphis May Fire'),
 (0.24000000000000002, 'Mumford & Sons'),
 (0.24000000000000002, 'Oh, Sleeper'),
 (0.24000000000000002, 'Sick Puppies'),
 (0.2, 'Avenged Sevenfold'),
 (0.2, 'In Flames'),
 (0.2, 'Pink Floyd'),
 (0.16000000000000003, 'Amon Amarth'),
 (0.16000000000000003, 'Arch Enemy'),
 (0.16000000000000003, 'DragonForce'),
 (0.16000000000000003, 'Hans Zimmer'),
 (0.16000000000000003, 'Howard Shore'),
 (0

6:

Die euklidische Distanz beschreibt die Ähnlichkeit zweier Objekte. Dabei wird die maximale Ähnlichkeit als 1 ausgedrückt und minimale Ähnlichkeit als 0. Aus der Liste ist ersichtlich, dass die Band "Korn" wahrscheinlich dem Nutzer 'BrunoJoS' gefallen würde.

Die Russel Rao Ähnlichkeit zeigt, dass die Bands "Five Finger Death Punch" und "Korn" dem Nutzer "BrunoJoS" am meisten gefallen könnten, weil die Nutzer, die "FFDP" und "Korn" hören ähnliche Top Artists haben.